<div align="center">
  <a target="_blank" href="https://colab.research.google.com/github/crowdcent/crowdcent-challenge/blob/main/docs/tutorials/client-cheatsheet.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
  </a>
</div>

### Set API Key

In [1]:
%load_ext dotenv
%dotenv

import crowdcent_challenge as cc
import polars as pl
import numpy as np
import os

CROWDCENT_API_KEY = os.environ["CROWDCENT_API_KEY"]

### Initialize the client

In [ ]:
client = cc.ChallengeClient(
    challenge_slug="hyperliquid-ranking",
)

### Get challenge and training datasets details

In [ ]:
client.get_challenge()

In [ ]:
client.list_training_datasets()

### Download current inference data

In [ ]:
client.get_inference_data("latest")

In [ ]:
import time

while True:  # keep checking until a period appears
    try:
        client.download_inference_data("current", "inference_data.parquet")
        print("✅ new inference period – file saved")
        break  # success → exit loop
    except cc.NotFoundError:  # no active period yet
        time.sleep(60)  # wait 60 s before trying again

In [ ]:
client.download_inference_data(
    release_date="latest", dest_path="inference_data.parquet"
)

In [ ]:
# Read in the inference data, in this case, just a universe
inference_data = pl.read_parquet("inference_data.parquet")
inference_data.head()

### Submit predictions

In [ ]:
# Replace this with your actual prediction logic

submission_data = inference_data.with_columns(
    [
        pl.Series("pred_10d", np.random.random(len(inference_data))).cast(pl.Float64),
        pl.Series("pred_30d", np.random.random(len(inference_data))).cast(pl.Float64),
    ]
)

# You can save this modified dataframe if needed
submission_data.write_parquet("submission.parquet")

In [ ]:
client.submit_predictions(df=submission_data)
client.submit_predictions(file_path="submission.parquet", slot=1)

In [ ]:
for slot in range(1, 6):
    client.submit_predictions(
        file_path="inference_data_with_predictions.parquet", slot=slot
    )

## Meta-model

In [11]:
client.download_meta_model(dest_path="meta_model.parquet")
pl.read_parquet("meta_model.parquet").head()

2025-06-05 02:33:58,718 - INFO - Downloading consolidated meta model for challenge 'hyperliquid-ranking' to meta_model.parquet
2025-06-05 02:33:59,265 - INFO - Successfully downloaded consolidated meta model to meta_model.parquet


id,pred_10d,pred_30d,release_date
str,f32,f32,"datetime[μs, UTC]"
"""PIXEL""",0.785101,0.415266,2025-06-03 14:58:09.696716 UTC
"""ADA""",0.286367,0.768597,2025-06-03 14:58:09.696716 UTC
"""RSR""",0.429568,0.512441,2025-06-03 14:58:09.696716 UTC
"""VINE""",0.429594,0.364257,2025-06-03 14:58:09.696716 UTC
"""GALA""",0.157936,0.82046,2025-06-03 14:58:09.696716 UTC
